In [3]:
import os
import pandas as pd
import tensorflow
import numpy as np
import transformers
import torch
import pandas as pd
import tensorflow as tf
#from datasets import load_dataset

print(transformers.__version__)
print(tf.__version__)



4.51.3
2.18.0


In [4]:
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

In [5]:
import tensorflow as tf
import os

if 'COLAB_TPU_ADDR' in os.environ:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
        tpu='grpc://' + os.environ['COLAB_TPU_ADDR']
    )
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("TPU initialized.")
else:
    print("TPU 환경이 아님. GPU 또는 CPU 사용 중입니다.")

TPU 환경이 아님. GPU 또는 CPU 사용 중입니다.


**데이타**

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC

/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC


In [8]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir)
train_data_path = os.path.join(data_dir, 'aug_plus_train.csv')

train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,기타 괴롭힘 대화,어이 코털 왜 왜 아놔됐고 너 매점가서 곰보빵 두개에 콜라하나 사와라 춘추 돈없어 ...
1,1,기타 괴롭힘 대화,쟤 먹는거 봐 토쏠려 저리 가줄래 꼴에 먹는거는 좋아해사지고 그래서 니가 돼지 인거...
2,2,일반 대화,이 의류 색깔 진짜 예쁘네요 네 저도 이 색깔이 너무 마음에 들어요 이 색깔이 잘 ...
3,3,직장 내 괴롭힘 대화,야 새꺄 일로와바 일병 보니까 너네가 이미 노래방 청소했는데 우리가 거기 담당이더라...
4,4,기타 괴롭힘 대화,야 넌 체육 수행평가 만점이라며 야 좋겠다 나도 다리 병신이었으면 얼마나 좋았을까 ...


In [9]:
print('전체 샘플수 :', (len(train_data)))
train_data.info()

전체 샘플수 : 14281
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14281 entries, 0 to 14280
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           14281 non-null  int64 
 1   class         14281 non-null  object
 2   conversation  14281 non-null  object
dtypes: int64(1), object(2)
memory usage: 334.8+ KB


In [10]:
# 클래스 라벨을 숫자로 변환
# 0, 1, 2, 3, 4
CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화']

class_dict = {}
for idx, class_name in enumerate(CLASS_NAMES):
    class_dict[class_name] = idx

train_data['label'] = train_data['class'].apply(lambda s: class_dict[s])

train_data.head()

,idx,class,conversation,label
0,0,기타 괴롭힘 대화,어이 코털 왜 왜 아놔됐고 너 매점가서 곰보빵 두개에 콜라하나 사와라 춘추 돈없어 ...,3
1,1,기타 괴롭힘 대화,쟤 먹는거 봐 토쏠려 저리 가줄래 꼴에 먹는거는 좋아해사지고 그래서 니가 돼지 인거...,3
2,2,일반 대화,이 의류 색깔 진짜 예쁘네요 네 저도 이 색깔이 너무 마음에 들어요 이 색깔이 잘 ...,4
3,3,직장 내 괴롭힘 대화,야 새꺄 일로와바 일병 보니까 너네가 이미 노래방 청소했는데 우리가 거기 담당이더라...,2
4,4,기타 괴롭힘 대화,야 넌 체육 수행평가 만점이라며 야 좋겠다 나도 다리 병신이었으면 얼마나 좋았을까 ...,3


In [11]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎ가-힣ㅏ-ㅣ?.!,]+", " ", sentence)

    sentence = sentence.strip()

    return sentence

In [12]:
train_data["conversation"] = [preprocess_sentence(s) for s in train_data["conversation"]]
train_data.head()

,idx,class,conversation,label
0,0,기타 괴롭힘 대화,어이 코털 왜 왜 아놔됐고 너 매점가서 곰보빵 두개에 콜라하나 사와라 춘추 돈없어 ...,3
1,1,기타 괴롭힘 대화,쟤 먹는거 봐 토쏠려 저리 가줄래 꼴에 먹는거는 좋아해사지고 그래서 니가 돼지 인거...,3
2,2,일반 대화,이 의류 색깔 진짜 예쁘네요 네 저도 이 색깔이 너무 마음에 들어요 이 색깔이 잘 ...,4
3,3,직장 내 괴롭힘 대화,야 새꺄 일로와바 일병 보니까 너네가 이미 노래방 청소했는데 우리가 거기 담당이더라...,2
4,4,기타 괴롭힘 대화,야 넌 체육 수행평가 만점이라며 야 좋겠다 나도 다리 병신이었으면 얼마나 좋았을까 ...,3


In [13]:
# train/test 분리
df = train_data

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['conversation'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    stratify=df['label']
)

**test data**

In [14]:
data_dir = os.path.join(current_dir)
test_data_path = os.path.join(data_dir, 'test.csv')


test_data = pd.read_csv(test_data_path)
test_data.head()

,idx,text
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1,t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...


In [15]:
# 한 문장의 길이를  num_tokens에 저장
num_tokens = [len(tokens) for tokens in train_data['conversation']]
# array로 변환
num_tokens = np.array(num_tokens)

# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다.
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

max_tokens = np.mean(num_tokens) + 2.5 * np.std(num_tokens)
maxlen = int(max_tokens)
print('train_sequences maxlen : ', maxlen)
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 maxlen 설정값 이내에 포함됩니다. ')

sequences_maxlen = maxlen
sequences_maxlen = 200

문장길이 평균 :  225.50654716056297
문장길이 최대 :  909
문장길이 표준편차 :  105.749061744247
train_sequences maxlen :  489
전체 문장의 0.9684195784608921%가 maxlen 설정값 이내에 포함됩니다. 


In [16]:
# 한 문장의 길이를  num_tokens에 저장
num_tokens = [len(tokens) for tokens in test_data['text']]
# array로 변환
num_tokens = np.array(num_tokens)

# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다.
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

max_tokens = np.mean(num_tokens) + 2.5 * np.std(num_tokens)
maxlen = int(max_tokens)
print('test_sequences maxlen : ', maxlen)
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 maxlen 설정값 이내에 포함됩니다. ')

test_maxlen = maxlen

문장길이 평균 :  213.626
문장길이 최대 :  722
문장길이 표준편차 :  101.13918194250931
test_sequences maxlen :  466
전체 문장의 0.972%가 maxlen 설정값 이내에 포함됩니다. 


**beomi/kcbert-base**

In [17]:
from transformers import TFBertForSequenceClassification

model_name = "beomi/kcbert-base"

# 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained(model_name)

#PyTorch로 학습된 beomi/kcbert-base 모델을 TensorFlow로 변환하여 사용
from transformers import TFBertModel

model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=len(class_dict),  from_pt=True)#  클래스 수에 맞게

# tokenizer의 vocab 사이즈 확인
tokenizer.vocab_size


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


30000

In [18]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=sequences_maxlen)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=sequences_maxlen)



In [19]:
# TensorFlow 데이터셋으로 변환
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

# 배치 처리 및 셔플
train_dataset = train_dataset.shuffle(1000).batch(16)
val_dataset = val_dataset.shuffle(1000).batch(16)

In [20]:
from transformers import AutoConfig


config = AutoConfig.from_pretrained(model_name)
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.51.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

In [21]:
# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])



In [22]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108918528 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3845      
                                                                 
Total params: 108922373 (415.51 MB)
Trainable params: 108922373 (415.51 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
for inputs, labels in train_dataset.take(1):
    print("입력:", inputs)
    print("라벨:", labels)

입력: {'input_ids': <tf.Tensor: shape=(16, 200), dtype=int32, numpy=
array([[    2, 10103, 18043, ...,     0,     0,     0],
       [    2, 27285,  2458, ...,     0,     0,     0],
       [    2,   634,  8362, ...,     0,     0,     0],
       ...,
       [    2,  9639, 13753, ...,     0,     0,     0],
       [    2,  8380, 14865, ...,     0,     0,     0],
       [    2,  2581,  9326, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(16, 200), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(16, 200), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int3

In [24]:
for inputs, labels in val_dataset.take(1):
    print("입력:", inputs)
    print("라벨:", labels)

입력: {'input_ids': <tf.Tensor: shape=(16, 200), dtype=int32, numpy=
array([[    2,  2207,  8362, ...,     0,     0,     0],
       [    2,  2523,  8314, ...,     0,     0,     0],
       [    2,  1125, 14047, ...,     0,     0,     0],
       ...,
       [    2,  1856,  4172, ...,     0,     0,     0],
       [    2,  2208,  4057, ...,     0,     0,     0],
       [    2,  8001,  1427, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(16, 200), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(16, 200), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int3

In [25]:
# 모델 학습
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1
)

714/714 [==============================] - 548s 706ms/step - loss: 0.2711 - accuracy: 0.9101 - val_loss: 0.1137 - val_accuracy: 0.9702


In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(val_dataset)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")


 50/179 [=======>......................] - ETA: 27:43 - loss: 1.6158 - accuracy: 0.1988

In [26]:
# 예측 함수
test_texts = test_data["text"].tolist()

test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=sequences_maxlen)

test_dataset = tf.data.Dataset.from_tensor_slices(dict(test_encodings)).batch(16)

logits = model.predict(test_dataset).logits

predictions = np.argmax(logits, axis=-1)

print(predictions)  # 예측된 클래스 인덱스 출력

32/32 [==============================] - 10s 225ms/step
[1 2 2 4 3 0 0 1 2 1 0 3 2 2 2 0 0 3 0 3 0 3 3 1 1 2 3 4 2 1 1 1 3 1 3 3 3
 0 0 1 0 1 2 1 3 2 0 3 4 0 1 3 1 2 2 3 3 3 3 3 1 3 0 3 2 0 3 3 4 1 3 2 3 2
 0 2 2 1 2 3 1 0 4 0 3 2 0 3 1 2 0 2 0 3 3 2 1 3 0 3 1 1 3 0 3 2 2 3 3 0 1
 0 1 2 1 1 3 2 2 1 2 3 1 1 1 1 1 0 1 3 3 3 2 3 1 0 3 0 3 3 0 3 1 3 2 0 0 0
 0 1 0 3 4 1 1 4 0 2 3 3 0 3 4 0 2 3 2 3 3 1 0 2 2 3 3 3 2 2 3 1 2 3 2 2 0
 3 2 0 1 0 0 3 1 3 1 3 3 1 0 3 2 0 3 2 0 3 2 1 2 2 0 1 3 2 1 3 3 0 3 0 1 4
 0 3 0 3 1 0 1 0 2 0 3 3 1 3 3 1 0 3 3 3 3 2 3 1 2 3 2 1 0 4 0 2 2 1 0 2 1
 2 1 3 0 3 2 3 3 3 3 1 0 3 0 2 1 2 2 3 2 2 1 0 3 2 2 2 1 2 1 3 1 1 3 3 2 3
 3 3 1 0 4 3 0 2 1 1 3 2 1 2 0 1 3 3 2 3 1 3 3 0 2 0 0 1 1 1 1 3 2 3 1 0 0
 3 1 3 3 1 3 3 0 2 0 0 1 2 2 1 0 0 0 3 1 1 1 3 3 3 0 1 3 3 1 1 2 3 3 2 0 1
 1 2 0 2 2 1 1 3 1 1 2 0 3 2 3 1 3 0 0 3 1 0 3 2 1 2 0 1 1 0 3 3 2 2 3 3 2
 3 2 1 3 3 0 2 4 0 3 1 1 3 1 0 0 3 3 1 1 2 2 0 3 3 1 0 1 3 0 3 3 2 3 0 2 3
 1 2 0 2 2 4 1 0 2 3 2 3 2 3 1 1 0 1 0 2 2 3

In [27]:
from keras.models import load_model

model.save('kcbert_model_300.keras')

/usr/local/lib/python3.11/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [28]:
# 예시: id와 예측된 클래스 저장
submission_df = pd.DataFrame({
    'idx': test_data['idx'],
    'class': predictions
})

display(submission_df)

# submission.csv로 저장
submission_df.to_csv('submission_kcbert_300.csv', index=False)

,idx,class
0,t_000,1
1,t_001,2
2,t_002,2
3,t_003,4
4,t_004,3
...,...,...
495,t_495,2
496,t_496,2
497,t_497,1
498,t_498,0
